In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from sk import predict
from glob import glob
from tqdm import tqdm

In [3]:
import malaya
import malaya_speech
import tensorflow as tf
malaya.__version__, tf.__version__

2021-10-04 21:00:29.921665: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
/home/ubuntu/.local/lib/python3.8/site-packages/malaya_boilerplate/frozen_graph.py:24: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0
  warnings.warn(


('4.7', '2.5.0')

In [4]:
# !pip3 install tensorflow-text==2.5 --no-deps
# !pip3 install tensorflow_hub

In [5]:
# super_super_tiny = malaya.true_case.transformer(model = 'super-tiny-t5', device = 'cpu:0')

In [6]:
aligner = malaya_speech.force_alignment.deep_aligner(model = 'conformer-transducer', device = 'cpu:0')

2021-10-04 21:00:34.286907: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-04 21:00:34.288457: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-10-04 21:00:34.321476: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-04 21:00:34.321514: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2021-10-04 21:00:34.321523: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincomel-desktop
2021-10-04 21:00:34.321625: I tensorflow/stream_executor/cuda/cuda_diagnos

In [8]:
wavs = glob('turki/*/*.wav')
len(wavs)

2621

In [61]:
def put(alignment, length, min_threshold = 0.5, min_threshold_fullstop = 0.8):
    r = []
    for no, row in enumerate(alignment):
        if no > 0:
            if alignment[no]['start'] - alignment[no-1]['end'] >= min_threshold:
                r.append(',')

        r.append(row['text'])
    if length - alignment[-1]['end'] >= min_threshold_fullstop:
        r.append('.')
    else:
        r.append(',')
    return r

In [65]:
results = {}
for wav in tqdm(wavs):
    with open(wav.replace('-wav-', '-text-') + '.txt') as fopen:
        data = fopen.read().lower()
#     data = predict(wav, decoder = 'v1')['texts'][0]
    y, sr = malaya_speech.load(wav)
    result = aligner.predict(y, data)
    if ' '.join([r['text'] for r in result['words_alignment']]) != data:
        continue
    o = put(result['words_alignment'], len(y) / sr)
    results[wav] = ' '.join(o)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2621/2621 [16:30<00:00,  2.65it/s]


In [64]:
import IPython.display as ipd
ipd.Audio('turki/output-wav-turki/janji-tinggal-janji-2.mp3-94.wav')

In [66]:
len(results), len(wavs)

(2166, 2621)

In [68]:
import json
with open('comma-fullstop-turki.json', 'w') as fopen:
    json.dump(results, fopen)